In [ ]:
from concurrent.futures import ProcessPoolExecutor

from timecraft_ai import DatabaseConnector
from timecraft_ai import TimeCraftModel

# TimeCraft - Forecasting with Oracle and PostgreSQL

This notebook demonstrates how to use TimeCraft to forecast product stock using data from Oracle and PostgreSQL databases. All code, comments, and outputs are standardized in English.

In [ ]:
def process_product(product_id):
    # Set up the database connector (PostgreSQL example)
    db_connector = DatabaseConnector(
        db_type="postgres",
        username="kubex_adm",
        password="abcdefg",
        dsn="192.168.100.15:5432/kubex_db"
    )
    
    # Read the SQL query template
    with open("data/CLI_X_PROD_CLI_SALE.sql.j2", "r") as file:
        query_template = file.read()
    
    # Render the template with the product ID
    query = query_template.format({"product_id": product_id})

    ts_model = TimeCraftModel(
        db_connector=db_connector,
        query=query.replace("{ product_id }", str(product_id)),
        date_column="DTNEG",
        value_columns=["SALDO_HISTORICO"],
        is_csv=False,
        periods=30
    )

    try:
        ts_model.run()
        # forecast = ts_model.get_forecast()
    except Exception as e:
        print(f"Error processing product {product_id}: {e}")
        return None
    
    print(f"Forecast for product {product_id} completed.")
    # print(forecast)

    # Save forecast results
    output_file = f"output/products_stock/forecast_stock_{product_id}.csv"
    ts_model.save_forecast(output_file)

    # Save forecast plots
    ts_model.save_plots(f"output/forecast_stock_{product_id}.png")

In [ ]:
def get_product_ids() -> list:
    query_products = """
    SELECT 
        P.CODPROD
    FROM 
        TGFPRO P
    WHERE 
        P.USOPROD = 'R'
        AND P.ATIVO = 'S'
        AND P.CODPROD IN(1,2,3,4,5,6,7,8,9,10)
    """

    # Example: Oracle connection
    db_connector = DatabaseConnector(
        db_type="oracle",
        username="sankhya",
        password="abcdefg",
        dsn="127.0.0.1:1521/orcl"
    )
    db_connector.connect()

    try:
        products_df = db_connector.execute_query(query_products)
    except Exception as e:
        print(f"Error querying products: {e}")
        return []

    db_connector.close()

    # List of product IDs
    product_ids = products_df["CODPROD"].tolist()
    return product_ids

In [ ]:
product_ids = get_product_ids()

with ProcessPoolExecutor(max_workers=4) as executor:
    executor.map(process_product, product_ids)

print("Processing completed.")